In [193]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

In [194]:
import spotipy
import spotipy.util as util

import os
os.environ['SPOTIPY_CLIENT_ID'] = "6051b4896eda4feca0e0e28bed145907"
os.environ['SPOTIPY_CLIENT_SECRET'] = "1d7c94e540d741758cc103db6e028a96"
os.environ['SPOTIPY_REDIRECT_URI'] = "https://localhost:8080"

token = util.prompt_for_user_token("31zahq3gf5zm3oansaxx76hufvpm")

# AUTHENTICATION
sp = spotipy.Spotify(auth=token)


In [195]:
genre_prediction_model = load_model("saved_model\\final_model")

In [196]:
key_map = {0: 'C',1: 'C#', 2: 'D',3: 'Eb',4: 'E',5: 'F',6: 'F#',7: 'G',8: 'Ab',9: 'A',10: 'Bb',11: 'B'}

In [197]:
def get_key_dict(key):
    d = {}
    lst = np.eye(12).tolist()[key]
    for i in range(len(lst)):
        d[key_map[i]] = lst[i]
    return d


In [198]:
def transform_key_dict(feat):
    key_dict = get_key_dict(feat["key"])
    feat.update(key_dict)
    del feat['key']
    del feat['B']
    return feat


In [199]:
def get_features(song_url):
    track = sp.track(song_url)
    features = sp.audio_features([track["id"]])[0]
    features.pop('type')
    features.pop('uri')
    features.pop('track_href')
    features.pop('analysis_url')
    features.pop('id')
    features.pop('liveness')
    features = transform_key_dict(features)
    features['major'] = features['mode']
    del features['mode']
    return features


In [200]:
genre_df = pd.read_csv('modified_genre_df.csv', sep = '\t').drop(["genre", "track_name", "artist","Unnamed: 0","Unnamed: 0.1", "liveness"], axis=1)

scaler = MinMaxScaler()

scaler.fit(genre_df)


MinMaxScaler()

In [201]:
genre_map = {
    'ROCK': 0,
    'POP': 1,
    'LOFI': 2,
    'CLASSICAL': 3,
    'RAP': 4
}

## Prediction of genre of a track:

In [202]:
def predict_genre(song_url, model):
    data = pd.Series(get_features(song_url))
    data = scaler.transform([data])
    predict_x = model.predict(data, verbose=False)
    classes_x = np.argmax(predict_x)
    my_inverted_dict = dict(map(reversed, genre_map.items()))
    return my_inverted_dict[classes_x], sp.track(song_url)['artists'][0]['name'], sp.track(song_url)['name']


In [229]:
genre, artist, song = predict_genre("https://open.spotify.com/track/2pscAWkm3jAxcrpBwOtQmS?si=f3c4c23f75d741fd", genre_prediction_model)

C:\Users\shail\anaconda3\Lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [230]:
print("Song:", song, "\nArtist:", artist, "\nGenre:", genre)

Song: VISIONS 
Artist: VALORANT 
Genre: POP
